# Lab 3: AgentCore Gateway でエージェントにツールを安全に接続

## 概要

このLabでは、Amazon Bedrock Gateway を使用して、組織内で利用可能なツールをカスタマーサポートエージェントと統合する方法を学びます。

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro) は、アプリケーションが大規模言語モデル（LLM）にツールとコンテキストを提供する方法を標準化するオープンプロトコルです。

[Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) を使用すると、開発者は API、Lambda 関数、既存のサービスを MCP 互換のツールに変換し、わずか数行のコードで Gateway エンドポイントを通じてエージェントに提供できます。


**ワークショップの流れ:**

- **Lab 1（完了）:** エージェントプロトタイプの作成 - 機能するカスタマーサポートエージェントを構築
- **Lab 2（完了）:** Memory による強化 - 会話コンテキストとパーソナライゼーションを追加
- **Lab 3（現在）:** Gateway と Identity によるスケーリング - エージェント間でツールを安全に共有
- **Lab 4:** 本番環境へのデプロイ - AgentCore Runtime を使用してオブザーバビリティを実現
- **Lab 5:** エージェントパフォーマンスの評価 - オンライン評価で品質を監視
- **Lab 6:** ユーザーインターフェースの構築 - 顧客向けアプリケーションを作成


### AgentCore Gateway とツール共有が重要な理由

現在の状態（Lab 1-2）: 各エージェントが独自のツールのコピーを持っています。これはスケーラブルではなく、以下の問題につながります：

- 異なるエージェント間でのコードの重複
- 一貫性のないツールの動作とメンテナンスのオーバーヘッド
- 集中化されたセキュリティやアクセス制御がない
- 複数のユースケースへのスケーリングが困難

このLabの後は、以下に対応できる集中化された再利用可能なツールを持つことになります：

- カスタマーサポートエージェント（現在のユースケース）
- セールスエージェント（同じ製品情報と顧客データが必要）
- 在庫エージェント（同じ製品情報と保証確認が必要）
- 返品処理エージェント（返品ポリシーと顧客プロファイルが必要）

およびその他のユースケース。

### AgentCore Identity による安全な認証の追加

さらに、AgentCore Gateway では、インバウンドとアウトバウンドの両方の接続を安全に認証する必要があります。[AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html) は、AWS サービスと Slack や Zoom などのサードパーティアプリケーション間でシームレスなエージェントアイデンティティとアクセス管理を提供し、Okta、Entra、Amazon Cognito などの標準的なアイデンティティプロバイダーをサポートします。このLabでは、AgentCore Gateway が AgentCore Identity と統合して、インバウンドおよびアウトバウンド認証による安全な接続を提供する方法を見ていきます。

インバウンド認証では、AgentCore Gateway は呼び出し時に渡された OAuth トークンを分析して、Gateway 内のツールへのアクセスを許可または拒否します。ツールが外部リソースにアクセスする必要がある場合、AgentCore Gateway は API Key、IAM、または OAuth Token によるアウトバウンド認証を使用して、外部リソースへのアクセスを許可または拒否できます。

インバウンド認可フローでは、エージェントまたは MCP クライアントが AgentCore Gateway 内の MCP ツールを呼び出し、OAuth アクセストークン（ユーザーの IdP から生成）を追加します。AgentCore Gateway は OAuth アクセストークンを検証し、インバウンド認可を実行します。

AgentCore Gateway で実行されているツールが外部リソースにアクセスする必要がある場合、OAuth は Gateway ターゲットのリソース資格情報プロバイダーを使用してダウンストリームリソースの資格情報を取得します。AgentCore Gateway は認可資格情報を呼び出し元に渡して、ダウンストリーム API へのアクセスを取得します。


## Lab 3 のアーキテクチャ

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*ウェブ検索ツールは、安全な Identity ベースのアクセス制御を備えた AgentCore Gateway に集中化されています。複数のエージェントとユースケースが同じツールを安全に共有できます。他のアプリケーション用に構築された `check_warranty()` ツールも再利用し、他のアプリケーション内で使用するための `web_search()` ツールも追加します。`get_product_info()`、`get_return_policy()`、`get_technical_support` は、カスタマーサポートのユースケースに特化しているため、ローカルツールとして残ります。*

### 主な機能
- **AWS Lambda 関数のシームレスな統合:** この例では、Amazon Bedrock AgentCore Gateway を使用して、既存の AWS Lambda 関数と統合し、アイテムの保証を確認したり、顧客プロファイルを取得したりする方法を示します。
- **インバウンド認証で Gateway エンドポイントを保護**: 有効な JWT トークンを提供するエージェントのみがツールを使用するためにエンドポイントに接続できます
- **MCP エンドポイントを使用するようにエージェントを設定**: エージェントは有効な JWT トークンを取得し、それを使用して AgentCore Gateway が提供する MCP エンドポイントに接続します

## 前提条件

* Python 3.12+
* AWS 認証情報が設定済み
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) で有効化された Anthropic Claude 3.7
* Lab 2 「カスタマーサポートエージェントへの Memory の追加」を完了
* AWS ワークショップアカウント内で作成される以下のリソース
    - AWS Lambda 関数 
    - AWS Lambda 実行 IAM ロール
    - AgentCore Gateway IAM ロール
    - AWS Lambda 関数が使用する DynamoDB テーブル
    - Cognito User Pool と User Pool Client

## ステップ 1: 必要なライブラリのインポート

In [ ]:
# Import libraries
import os
import sys
import boto3

from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from lab_helpers.utils import (
    get_or_create_cognito_pool,
    put_ssm_parameter,
    get_ssm_parameter,
    load_api_spec,
)


sts_client = boto3.client("sts")

# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ ライブラリのインポートに成功しました！")

## ステップ 2: 既存の顧客データにアクセスするツールをエージェントに提供
AgentCore Gateway は、3つの主要な方法でエージェントツールの統合を簡素化します：

ユニバーサル MCP サポート: AgentCore Gateway の MCP 標準を通じてツールを公開することで、任意のエージェントフレームワークと即座に互換性を持たせる

シンプルな REST 統合: 既存の REST サービスを AgentCore Gateway ターゲットとして追加するだけで、エージェントツールに変換

Lambda の柔軟性: 任意の API を呼び出せる Lambda 関数を MCP エンドポイントとして公開 - ここでは保証ステータスをチェックする関数で実演

AgentCore Gateway は Lambda コンテキストに呼び出すツールの名前を入力し、ツールに渡されるパラメータは Lambda イベントで提供されます：

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda 関数](./prerequisite/lambda/python/lambda_function.py)

```
def lambda_handler(event, context):
    if get_tool_name(event) == "check_warranty_status":
        serial_number = get_named_parameter(event=event, name="serial_number")
        customer_email = get_named_parameter(event=event, name="customer_email")

        warranty_status = check_warranty_status(serial_number, customer_email)
        return {"statusCode": 200, "body": warranty_status}
```

## ステップ 3: ウェブ検索ツールを MCP に変換
AgentCore Gateway を使用して MCP サーバーを開発しているので、複数のエージェントで使用すると思われるツールを MCP 化できます。そのようなツールの1つが、Lab 1 で構築したウェブ検索ツールのようなものかもしれません。そこで、Lab 1 のウェブ検索ツールも AgentCore Gateway 内の Lambda ツールに変換しました：

[ウェブ検索 Lambda](./prerequisite/lambda/python/web_search.py)
```
from ddgs import DDGS


def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """ウェブで最新情報を検索します。
    
    Args:
        keywords (str): 検索クエリのキーワード。
        region (str): 検索リージョン: wt-wt, us-en, uk-en, ru-ru など。
        max_results (int): 返す結果の最大数。
        
    Returns:
        検索結果を含む辞書のリスト。
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "結果が見つかりませんでした。"
    except Exception as e:
        return f"検索エラー: {str(e)}"


print("✅ ウェブ検索ツール準備完了")
```

## ステップ 4: 関数定義メタデータの作成
最後に、Lambda 関数によって実装されるツールを記述するツールスキーマを記述する必要があります。

このファイルは既に [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json) で定義されています

```
[
    {
        "name": "check_warranty_status",
        "description": "製品のシリアル番号を使用して保証ステータスを確認し、オプションでメールで確認します",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {
                    "type": "string"
                },
                "customer_email": {
                    "type": "string"
                }
            },
            "required": [
                "serial_number"
            ]
        }
    },
    {
        "name": "web_search",
        "description": "DuckDuckGo を使用してウェブで最新情報を検索します",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {
                    "type": "string",
                    "description": "検索クエリのキーワード"
                },
                "region": {
                    "type": "string",
                    "description": "検索リージョン（例: us-en, uk-en, ru-ru）"
                },
                "max_results": {
                    "type": "integer",
                    "description": "返す結果の最大数"
                }
            },
            "required": [
                "keywords"
            ]
        }
    }
]
```

## ステップ 5: AgentCore Gateway の作成

次に、Lambda 関数を MCP 互換のエンドポイントとして公開する AgentCore Gateway を作成しましょう。

ツールを呼び出す権限のある呼び出し元を検証するには、インバウンド認証を設定する必要があります。

インバウンド認証は、MCP サーバーの標準である OAuth 認可を使用して動作します。OAuth では、クライアントアプリケーションは Gateway を使用する前に OAuth オーソライザーで認証する必要があります。クライアントは実行時に使用されるアクセストークンを受け取ります。

OAuth ディスカバリサーバーとクライアント ID を指定する必要があります。ワークショップで提供される CloudFormation は、Cognito UserPool と UserPoolClient を既にプロビジョニングしており、ディスカバリ URL とクライアント ID を専用の SSM パラメータに保存しています。

In [ ]:
gateway_name = "customersupport-gw"

cognito_config = get_or_create_cognito_pool(refresh_token=True)
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

try:
    # create new gateway
    print(f"リージョン {REGION} に名前 {gateway_name} で Gateway を作成しています")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_name", gateway_name)
    put_ssm_parameter(
        "/app/customersupport/agentcore/gateway_arn", create_response["gatewayArn"]
    )
    put_ssm_parameter(
        "/app/customersupport/agentcore/gateway_url", create_response["gatewayUrl"]
    )
    print(f"✅ Gateway が正常に作成されました。ID: {gateway_id}")

except Exception:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    print(f"既存の Gateway が見つかりました。ID: {existing_gateway_id}")

    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway["id"]

## ステップ 6: Lambda 関数ターゲットの追加
次に、[prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json) で以前に定義した関数定義を使用して、Agent Gateway 内に Lambda ターゲットを作成します。これにより、Gateway がホストするツールが定義されます。

Gateway では、複数のターゲットを Gateway にアタッチでき、いつでも Gateway にアタッチされたターゲット/ツールを変更できます。各ターゲットは独自の資格情報プロバイダーを持つことができますが、Gateway はエージェント用のすべての関連ツールに無数の API を通じてアクセスできる単一の MCP URL になります。

In [ ]:
try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API 仕様ファイルが見つかりません: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)

    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter(
                    "/app/customersupport/agentcore/lambda_arn"
                ),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway ターゲットが作成されました: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ Gateway ターゲットの作成中にエラーが発生しました: {str(e)}")

## ステップ 7: 新しい MCP ベースのツールをサポートエージェントに追加
ここでは、Strands SDK の MCPClient に Cognito からの認証トークンを統合して、Strands Agent と統合するための MCP Server オブジェクトを作成します
### ステップ 7.1: 安全な MCP クライアントオブジェクトのセットアップ

In [ ]:
print(f"Gateway エンドポイント - MCP URL: {gateway['gateway_url']}")
# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway["gateway_url"],
        headers={"Authorization": f"Bearer {cognito_config['bearer_token']}"},
    )
)

## ステップ 7.2: MCP クライアントを使用してエージェントでツールにアクセス
次に、構築した AgentCore Gateway と以前のLabのリソースを使用して Strands Agent を作成します。エージェントは、Strands Agent を介したローカルツールと AgentCore Gateway を介した MCP ツールの組み合わせを使用するようになりました

In [ ]:
from lab_helpers.lab1_strands_agent import (
    get_product_info,
    get_return_policy,
    get_technical_support,
    SYSTEM_PROMPT,
)

import uuid
from lab_helpers.lab2_memory import create_or_get_memory_resource
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig, RetrievalConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

memory_id = create_or_get_memory_resource()

SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_001"

memory_config = AgentCoreMemoryConfig(
        memory_id=memory_id,
        session_id=str(SESSION_ID),
        actor_id=CUSTOMER_ID,
        retrieval_config={
            "support/customer/{actorId}/semantic": RetrievalConfig(top_k=3, relevance_score=0.2),
            "support/customer/{actorId}/preferences": RetrievalConfig(top_k=3, relevance_score=0.2)
        }
    )

# Initialize the Bedrock model
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION,
)


def create_agent(prompt):
    try:
        with mcp_client:
            tools = [
                get_product_info,
                get_return_policy,
                get_technical_support,
            ] + mcp_client.list_tools_sync()

            # Create the customer support agent
            agent = Agent(
                model=model,
                tools=tools,
                system_prompt=SYSTEM_PROMPT,
                session_manager=AgentCoreMemorySessionManager(memory_config, REGION),
            )
            response = agent(prompt)
            return response
    except Exception as e:
        raise e


print("✅ カスタマーサポートエージェントが正常に作成されました！")

## ステップ 8: 既存の API への MCP ツールアクセスでエージェントをテスト

サンプルクエリでエージェントをテストして、すべての機能が正しく動作することを確認しましょう。出力には5つのツールが表示されるはずです。

In [ ]:
test_prompts = [
    # Warranty Checks
    "List all of your tools",
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?",
    "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333.",
    "What are the warranty support guidelines?",
    "How can I fix Lenovo Thinkpad with a blue screen",
    "Tell me detailed information about the technical documentation on installing a new CPU",
]


# Function to test the agent
def test_agent_responses(prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nテストケース {i}: {prompt}")
        print("-" * 50)
        try:
            response = create_agent(prompt)
            print(response)
        except Exception as e:
            print(f"エラー: {str(e)}")
        print("-" * 50)


# Run the tests
test_agent_responses(test_prompts)

print("\\n✅ 基本テスト完了！")

### おめでとうございます！


Lab 3: AgentCore Gateway でエージェントにツールを安全に接続を正常に完了しました！

達成したこと：

##### ツールの集中化と再利用性：

- ウェブ検索をローカルツールから集中化された AgentCore Gateway に移行
- 既存のエンタープライズ Lambda 関数（保証確認、顧客プロファイル）を統合
- 複数のエージェントタイプがアクセスできる共有ツールインフラストラクチャを作成

##### エンタープライズグレードのセキュリティ：

- Cognito 統合による JWT ベースの認証を実装
- Gateway アクセス用の安全なインバウンド認可を設定
- ツール使用のための Identity ベースのアクセス制御を確立

##### スケーラブルなアーキテクチャ基盤：

- 複数のユースケース（カスタマーサポート、セールス、返品処理）に対応する再利用可能なツールを構築
- 異なるエージェント間でのコードの重複を排除
- ツールの更新とメンテナンスのための集中管理を作成

##### 現在の制限事項（次のLabで修正します！）：

- **ローカル開発環境** - まだノートPC上で実行されており、本番環境対応ではない
- **限定的なオブザーバビリティ** - エージェントの動作とパフォーマンスの包括的な監視がない
- **手動スケーリング** - 増加した負荷や複数の同時ユーザーを自動的に処理できない

##### 次のステップ：Lab 4 - AgentCore Runtime で本番環境にデプロイ

Lab 4 では、プロトタイプを以下を備えた本番環境対応システムに変換します：

- スケーラブルなエージェントデプロイのための AgentCore Runtime
- メトリクス、ロギング、トレーシングによる包括的なオブザーバビリティ
- 実世界のトラフィックを処理するための自動スケーリング機能

### リソース
- [Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)
- [Strands Agents ドキュメント](https://github.com/strands-agents/sdk-python)
- [公式カスタマーサポートサンプル](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)